## Satellite Data

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:

    - From MAPBIOMAS:
        - secondary forest age
        - land use land cover
        - fire
    - From ESA CCI Biomass:
        - biomass (Mg C/hectare)

Outputs:

    - From MAPBIOMAS:
        - last observed land use type
        - number of years under each land use type
        - number of fires
        - time since last fire
        - fallow period length



In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
data_folder, roi, years, first_year, last_year = global_variables()


## Removing pixels with undesired land use categories

Some land use categories are not relevant to the model (such as rocky surfaces or mangroves)

All pixels with **at least one observation of the undesired land use history** are used to make a mask, to leave behind only pixels with occurrences of only desired land use types.


Land use types we are interested in:

    3 = forest
    15 = pasture
    39 = soy
    20 = sugar cane
    21 = mosaic of uses
    40 = rice
    62 = cotton
    41 = other temporary crop
    46 = coffee
    47 = citrus
    35 = palm oil
    48 = other perennial crop
    9 = forest plantationantation

In [3]:
# For each band, convert pixels with desired land use types to 1 - undesired types to zero
def remap_band(band_name):
    # List the categories that are DESIRED to be maintained
    desired_values = ee.List([3, 6, 15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21])
    mask_all_ones = ee.List.repeat(1, desired_values.size())

    band = lulc_raw.select(ee.String(band_name))
    new_band = band.remap(desired_values, mask_all_ones, 0)
    return new_band.rename(ee.String(band_name))

def desired_lulc():

    # import ages from MapBiomas
    age = ee.Image(
        "projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_secondary_vegetation_age_v2"
    ).select("secondary_vegetation_age_2020")

    # Load images from MapBiomas Collection 8 for Land Use Land Cover and Burned Area
    lulc_all = (
        ee.Image(
            "projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1"
        )
        .select([f"classification_{year}" for year in range(first_year, last_year + 1)])
        .byte()
        .rename([str(year) for year in range(first_year, last_year + 1)])
    )

    # restricting to only the land use in pixels classified as secondary forests in 2020:
    lulc_raw = lulc_all.updateMask(age)

    fire = (
        ee.Image(
            "projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1"
        )
        .select([f"burned_coverage_{year}" for year in range(first_year, last_year)])
        .byte()
        .rename([str(year) for year in range(first_year, last_year)])
        .updateMask(age)
    )


    # Map the function over the band names
    remapped_image = lulc_raw.bandNames().map(remap_band)
    # make mask by adding all pixels that add up to the total number of years (all pixels with desired categories)
    remapped_image = ee.ImageCollection(remapped_image).toBands()
    desired_mask = remapped_image.reduce("sum").eq(lulc_raw.bandNames().size().getInfo())

    age = age.updateMask(desired_mask).rename("age")
    lulc = lulc_raw.updateMask(desired_mask)
    fire = fire.updateMask(desired_mask)

    return age, lulc, fire

age, lulc, fire = desired_lulc()

In [ ]:
# This collection is not publicly accessible. To sign up for access,
# please see https://developers.planet.com/docs/integrations/gee/nicfi
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')
basemap = nicfi.filter(ee.Filter.date('2018-03-01', '2018-07-01')).first()
vis_planet = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}

celso_ages = ee.Image("projects/amazon-forest-regrowth/assets/celso_ages/file")

In [47]:
# Load the image collections
Transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(age.geometry())
AnnualChanges = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(age.geometry())

# Define regrowth and degraded conditions
regrowth = Transition.gte(31).And(Transition.lte(33))  # include classes 63, 64 if you want to add regrowing mangroves

# Initialize AgeRegrowth and AgeDegraded
AgeRegrowth = ee.Image.constant(0)

# Calculate AgeRegrowth
for i in range(1990, 2020):
    year = 'Dec' + str(i)
    AnnualChangesYear = AnnualChanges.select(year)
    condition = AnnualChangesYear.eq(4).And(regrowth) # were regrowing then AND are regrowing now
    AgeRegrowth = AgeRegrowth.add(condition.eq(1))

age_eu = AgeRegrowth.selfMask()



In [66]:
mature = AnnualChanges.select('Dec2020').eq(1).selfMask()

Map.addLayer(mature, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Mature Forests')

## Biomass - Export age_agbd

Biomass data is in hectares, but mapbiomas data is 30m resolution.

To deal with edge pixels, we reproject biomass values to 10m resolution and then reaggregate to 30m by using the mean (so there is a buffer for land use pixels caught in between two biomass values).

In [54]:
# biomass for 2020 comes from CCI Biomass
biomass_all = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_all.projection().getInfo()

# Reproject to 10m
biomass = biomass_all.reproject(crs=age.projection(), scale=10)
# Reaggregate to 30m (mean value)
biomass = biomass.reduceResolution(reducer=ee.Reducer.mean()).reproject(
    crs = age.projection()
)

age_mapbiomas = age.updateMask(age_eu).rename("age_mapbiomas")
age_eu = age_eu.updateMask(age_mapbiomas).rename("age_eu")
age_mapbiomas = age_mapbiomas.updateMask(age_eu).rename("age_mapbiomas")

biomass = biomass.updateMask(age_mapbiomas).int16().rename("agbd")


age_agbd = age_mapbiomas.updateMask(biomass).addBands([biomass, age_eu]).addBands(age_mapbiomas.pixelLonLat().float())

if export_age_agbd:
    export_image(age_agbd, "age_agbd_eu_mapb", scale = 30)

In [64]:
categorical = ee.Image(f"{data_folder}/categorical")
# Create masks for biomes 1 and 4
biome_mask = categorical.select('biome').eq(4)#.Or(categorical.select('biome').eq(4)).Or(categorical.select('biome').eq(6))
categorical = categorical.updateMask(biome_mask)

cwd = ee.Image(f"{data_folder}/raw/cwd_chave")

unified_data = age_agbd.addBands([categorical, cwd]).updateMask(age_mapbiomas)

unified_data_sampled = unified_data.stratifiedSample(
    numPoints = 30000, classBand = 'biome', region = age_agbd.geometry()
)

task = ee.batch.Export.table.toDrive(
    collection = unified_data_sampled, description = "mapbiomas_eu_atl", fileFormat = 'CSV'
)
task.start()


## Remove isolated pixels - Export one_hectare_mask

In the map, there were isolated pixels, often around the edges of forest patches. These would likely be due to misclassification, or follow different behaviors due to edge effects.

To avoid this issue, a kernel is applied here to include only secondary forest patches that are mostly surrounded by other secondary forest pixels.

In [55]:
# convert non-forest pixels from NA to zero
# check what is the most frequent value within each hectare - if it's zero, it means the pixel is surrounded by non-forest cover
one_hectare_mask = age.unmask(0).focalMode(kernelType = "circle", radius = 100, units = "meters").selfMask()

if export_age_area_mask:
    export_image(one_hectare_mask, "one_hectare_mask", scale=30)